# Задача

Проверить поднятый сервис, привести примеры использования API.

# Импорты

In [1]:
import os
import urllib.parse
from pprint import pprint

import requests
import pandas as pd
from sklearn.metrics import roc_auc_score, plot_roc_curve

from module.prepare_data import load_dataset

# Данные

In [2]:
test_data_path = '../data/test.csv'

test_data = load_dataset(test_data_path)

In [3]:
test_data

,text,label
0,"ток чет не совпадает то, что они пишут, с тем,...",0
1,как я твою аву-то уберу?\nа без авы Дени теряе...,1
2,"Короче, Гил не даёт параллельно выполняться чи...",0
3,"Зачем делать remove, просто перезаписывать кон...",0
4,"в линуксах, говорят, всё экспортится само, что...",1
...,...,...
1236,Хорош интерпретацией. Плох при сильно несбала...,1
1237,придумул кое что другое :)\nПри добавлении\nda...,0
1238,"Нет, спасибо, я лучше на своем дистре посижу, ...",0
1239,"Да там и нет ничего умнее, там просто туча опт...",1


In [4]:
list_of_data = [{'text': text} for text in test_data['text'].values]

In [5]:
list_of_data[:5]

[{'text': 'ток чет не совпадает то, что они пишут, с тем, что я получаю  :D'},
 {'text': 'как я твою аву-то уберу?\\nа без авы Дени теряется смысл'},
 {'text': 'Короче, Гил не даёт параллельно выполняться чисто питоновскому коду, но многое нативное же'},
 {'text': 'Зачем делать remove, просто перезаписывать контент под этим же именем'},
 {'text': 'в линуксах, говорят, всё экспортится само, что нужно (не проверял)'}]

# Сервис

In [6]:
url = 'http://0.0.0.0:5000/'

### Проверка связи

In [7]:
ping_url = urllib.parse.urljoin(url, '/ping')
response = requests.get(ping_url)
response.raise_for_status()

print(response.text)

pong


### Проверка работы

In [8]:
model_url = urllib.parse.urljoin(url, '/predict')

#### Одиночный пример

In [9]:
%%time
api_json = list_of_data[0]
pprint(api_json)
response = requests.post(model_url, json=api_json)
response.raise_for_status()

pprint(response.json())

{'text': 'ток чет не совпадает то, что они пишут, с тем, что я получаю  :D'}
0.6859662565040332
CPU times: user 2.73 ms, sys: 0 ns, total: 2.73 ms
Wall time: 5.67 ms


#### Одиночный пример как список

In [10]:
%%time
api_json = list_of_data[:1]
pprint(api_json)
response = requests.post(model_url, json=api_json)
response.raise_for_status()

pprint(response.json())

[{'text': 'ток чет не совпадает то, что они пишут, с тем, что я получаю  :D'}]
[0.6859662565040332]
CPU times: user 2.91 ms, sys: 0 ns, total: 2.91 ms
Wall time: 5.74 ms


#### Список

In [11]:
%%time
api_json = list_of_data[:10]
pprint(api_json)
response = requests.post(model_url, json=api_json)
response.raise_for_status()

pprint(response.json())

[{'text': 'ток чет не совпадает то, что они пишут, с тем, что я получаю  :D'},
 {'text': 'как я твою аву-то уберу?\\nа без авы Дени теряется смысл'},
 {'text': 'Короче, Гил не даёт параллельно выполняться чисто питоновскому '
          'коду, но многое нативное же'},
 {'text': 'Зачем делать remove, просто перезаписывать контент под этим же '
          'именем'},
 {'text': 'в линуксах, говорят, всё экспортится само, что нужно (не проверял)'},
 {'text': '"содержит таблицу с идентификатором и значением таргета" см. скрин'},
 {'text': 'я ни разу не писатель, на работе то отчеты за меня пишут другие'},
 {'text': 'Очевидно что биннинг нужен длялогита чтобы нелинейную зависимость '
          'преобразраать в линейную от логорифма....'},
 {'text': 'Попытка подогнать реальность под идеал, и тем самым форсировать '
          'наступление идеала'},
 {'text': 'Не, там становится видно одного и того же пациента'}]
[0.6859662565040332,
 0.5562194849154555,
 0.027762636803320796,
 0.07279617991625546

#### Неправильное поле

In [12]:
%%time
api_json = {'comment': 'test'}
pprint(api_json)

response = requests.post(model_url, json=api_json)

print(response.text)

{'comment': 'test'}
{
  "code": 400, 
  "message": "Data should have field 'text'."
}

CPU times: user 3.4 ms, sys: 0 ns, total: 3.4 ms
Wall time: 4.01 ms


#### Неправильный тип данных

In [13]:
%%time
api_json = 'test'
pprint(api_json)

response = requests.post(model_url, json=api_json)

print(response.text)

'test'
{
  "code": 400, 
  "message": "Data should be dict or list of dicts."
}

CPU times: user 0 ns, sys: 2.45 ms, total: 2.45 ms
Wall time: 3.15 ms


In [14]:
%%time
api_json = [[{'text': 'test'}]]
pprint(api_json)

response = requests.post(model_url, json=api_json)

print(response.text)

[[{'text': 'test'}]]
{
  "code": 400, 
  "message": "Data should be dict or list of dicts."
}

CPU times: user 0 ns, sys: 2.64 ms, total: 2.64 ms
Wall time: 2.98 ms


## Проверка сервиса на всём тесте

In [15]:
%%time
response = requests.post(model_url, json=list_of_data)
response.raise_for_status()

prediction = response.json()

CPU times: user 3.66 ms, sys: 2.02 ms, total: 5.68 ms
Wall time: 1.25 s


In [16]:
test_label = test_data['label']

In [17]:
roc_auc_score(test_label, prediction)

0.8855504018321223